# Lab 4


## Problem 1: Warm up. Grid Search CV


In [0]:
## https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

print(__doc__)

# Loading the Digits dataset
digits = datasets.load_digits()
digits
# To apply an classifier on this data, we need to flatten the image, to
# turn the data in a (samples, feature) matrix:
n_samples = len(digits.images)
X = digits.images.reshape((n_samples, -1))
y = digits.target

print("X:\n", X)
print("Y:\n,", y)

# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=0)


# Set the parameters by cross-validation
# tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
#                      'C': [1, 10, 100, 1000]},
#                     {'kernel': ['linear'], 'C': [1, 10, 100, 1000]},
#                    {'kernel': ['poly'], 'C': [1e-3, 1e-4, 1, 10, 100, 1000]}]
tuned_parameters = [{'kernel': ['rbf', 'linear', 'poly'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]}]
scores = ['precision', 'recall']
for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(SVC(gamma='auto'), tuned_parameters, cv=5,
                       scoring='%s_macro' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Note the problem is too easy: the hyperparameter plateau is too flat and the
# output model is the same for precision and recall with ties in quality.

Automatically created module for IPython interactive environment
X:
 [[ 0.  0.  5. ...  0.  0.  0.]
 [ 0.  0.  0. ... 10.  0.  0.]
 [ 0.  0.  0. ... 16.  9.  0.]
 ...
 [ 0.  0.  1. ...  6.  0.  0.]
 [ 0.  0.  2. ... 12.  0.  0.]
 [ 0.  0. 10. ... 12.  1.  0.]]
Y:
, [0 1 2 ... 8 9 8]
# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}

Grid scores on development set:

0.986 (+/-0.016) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.975 (+/-0.014) for {'C': 1, 'gamma': 0.001, 'kernel': 'linear'}
0.987 (+/-0.020) for {'C': 1, 'gamma': 0.001, 'kernel': 'poly'}
0.959 (+/-0.029) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.975 (+/-0.014) for {'C': 1, 'gamma': 0.0001, 'kernel': 'linear'}
0.902 (+/-0.040) for {'C': 1, 'gamma': 0.0001, 'kernel': 'poly'}
0.988 (+/-0.017) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.975 (+/-0.014) for {'C': 10, 'gamma': 0.001, 'kernel': 'linear'}
0.987 (+/-0.020) for {'C': 10, 

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


## Problem 2: Lasso, Forward Selection and Cross Validation.

In [0]:
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
import pandas as pd
from pandas import DataFrame, Series
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
sns.set(style='ticks', palette='Set2')
%matplotlib inline

In [0]:
# Problem 2.1
# generate some sparse data to play with
np.random.seed(7)

n_samples, n_features = 100, 200
X = np.random.randn(n_samples, n_features)

k = 5
# beta generated with k nonzeros
#coef = 10 * np.random.randn(n_features)
coef = 10 * np.ones(n_features)
inds = np.arange(n_features)
print("coef\n", coef)
print("inds\n", inds)

np.random.shuffle(inds)
coef[inds[k:]] = 0  # sparsify coef
print("ind\n", inds)
print("coef after shuffling\n", coef)
y = np.dot(X, coef).reshape(n_samples, 1)

# add noise
y += 0.01 * np.random.randn(n_samples, 1)
print("actual y\n", y)


In [0]:
# manually implement forward selection
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

error_list = []
r_sq_list = []
features_added_idx = []
features_added_arr = np.empty((100, 0))

# find k non-zero beta values
for i in range(0, k):
    # find most significant beta out of all the features
    for feature in range(0, n_features):
      if feature in features_added_idx:
        error_list.append(float("inf"))
        continue  # skip if feature was already added
        
      n_samples = X.shape[0]
      X_subset = np.append(features_added_arr, X[:, feature].reshape(-1, 1), axis=1)  # get only one feature
      
      # perform Linear Regression with LSE
      model = LinearRegression()
      model.fit(X_subset, y)
      
      # find error and r^2 value
      y_pred = model.predict(X_subset)
      error = np.sum(np.square(y-y_pred))
      # print(f"error for feature {feature} is {error}")
      error_list.append(error)
      
      r_sq = model.score(X_subset, y)
      # print(f"r^2 score is {r_sq}")
      r_sq_list.append(r_sq)
      
      
    # add feature with lowest error
    min_error = min(error_list)
    feature_idx = error_list.index(min_error)
    features_added_idx.append(feature_idx)
    features_added_arr = np.append(features_added_arr, X[:, feature_idx].reshape(-1, 1), axis=1)
    
    error_list.clear()
    r_sq_list.clear()
    
print("features added in order using forward regression")    
print(features_added_idx)
    

features added in order using forward regression
[18, 15, 51, 78, 34]


In [0]:
# Problem 2.3
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
import warnings
warnings.filterwarnings("ignore")

# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=0)

# Set the parameters by cross-validation
tuned_parameters = [{'alpha': [1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]}]
scores = ['neg_mean_squared_error', 'r2']
for score in scores:
    print(f'# Tuning hyper-parameters for {score}')
    print()

    clf = GridSearchCV(Lasso(), tuned_parameters, cv=5,
                       scoring=score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print(f"{mean} (+/-{std*2}) for {params}")
        
    y_true, y_pred = y_test, clf.predict(X_test)

# Tuning hyper-parameters for neg_mean_squared_error

Best parameters set found on development set:

{'alpha': 0.01}

Grid scores on development set:

-709.6880914478731 (+/-629.7730951190001) for {'alpha': 0.0001}
-380.93023150315565 (+/-375.45492098712674) for {'alpha': 0.001}
-0.0009572951765671724 (+/-0.0010602438089130483) for {'alpha': 0.01}
-0.08016318292552004 (+/-0.09501160006079687) for {'alpha': 0.1}
-8.006704642092345 (+/-9.48642981748009) for {'alpha': 1}
-478.4405111441862 (+/-498.3533243100588) for {'alpha': 10}
-549.2956101694538 (+/-507.0252431973097) for {'alpha': 100}
# Tuning hyper-parameters for r2

Best parameters set found on development set:

{'alpha': 0.01}

Grid scores on development set:

-0.789341662271314 (+/-2.109384712511707) for {'alpha': 0.0001}
0.14833073021423782 (+/-0.6651766213136853) for {'alpha': 0.001}
0.9999982111012149 (+/-9.592262842603306e-07) for {'alpha': 0.01}
0.9998596656294025 (+/-9.7545209422529e-05) for {'alpha': 0.1}
0.985934921088574

We will use the hyperparameter alpha = 0.01.

In [0]:
# Problem 2.4
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
import warnings
warnings.filterwarnings("ignore")

# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=0)

# Set the parameters by cross-validation
tuned_parameters = [{'alpha': [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2, 1e-1]}]
folds = [2, 3, 4, 5, 7, 9, 10, 12, 15]
# scores = ['r2', 'neg_mean_squared_error']
scores = ['r2']
for fold in folds:
    for score in scores:
        print(f'##### Tuning hyper-parameters for {score}')
        print()

        clf = GridSearchCV(Lasso(), tuned_parameters, cv=fold,
                           scoring=score)
        clf.fit(X_train, y_train)

        print(f"Best parameters set found on development set: {clf.best_params_}, has {fold} folds.")
        print()
        print("Grid scores on development set:")
        print()
        means = clf.cv_results_['mean_test_score']
        stds = clf.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf.cv_results_['params']):
            print(f"{mean:.4f} (+/-{std*2:.4f}) for {params}")

        y_true, y_pred = y_test, clf.predict(X_test)

##### Tuning hyper-parameters for r2

Best parameters set found on development set: {'alpha': 0.1}, has 2 folds.

Grid scores on development set:

-1.3960 (+/-1.7796) for {'alpha': 1e-05}
-1.3959 (+/-1.7795) for {'alpha': 5e-05}
-1.3957 (+/-1.7794) for {'alpha': 0.0001}
-0.9718 (+/-1.2041) for {'alpha': 0.0005}
-0.6798 (+/-0.6721) for {'alpha': 0.001}
0.3938 (+/-0.1389) for {'alpha': 0.005}
0.6992 (+/-0.2962) for {'alpha': 0.01}
0.9956 (+/-0.0086) for {'alpha': 0.05}
0.9997 (+/-0.0003) for {'alpha': 0.1}
##### Tuning hyper-parameters for r2

Best parameters set found on development set: {'alpha': 0.05}, has 3 folds.

Grid scores on development set:

-0.6647 (+/-1.0177) for {'alpha': 1e-05}
-0.6645 (+/-1.0176) for {'alpha': 5e-05}
-0.6643 (+/-1.0174) for {'alpha': 0.0001}
-0.2307 (+/-0.6170) for {'alpha': 0.0005}
-0.0045 (+/-0.5056) for {'alpha': 0.001}
0.8438 (+/-0.0464) for {'alpha': 0.005}
0.9985 (+/-0.0042) for {'alpha': 0.01}
1.0000 (+/-0.0000) for {'alpha': 0.05}
0.9998 (+/-0.0001

The value of the hyperparameter varies. It seems that alpha goes down, generally, as there are more folds. This is probably due to overfitting on the data.

In [0]:
# Problem 2.5
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso, LassoCV

alphas = [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2, 1e-1]
folds = [2, 3, 4, 5, 7, 9, 10, 12, 15]

for fold in folds:
    lasso_cv = LassoCV(alphas=alphas, cv=fold).fit(X, y)
    print(f'Lasso CV has R^2 of {lasso_cv.score(X, y):.7f}. Alpha is: {lasso_cv.alpha_}. # folds are: {fold}.')


Lasso CV has R^2 of 1.0000000. Alpha is: 1e-05. # folds are: 2.
Lasso CV has R^2 of 0.9999996. Alpha is: 0.001. # folds are: 3.
Lasso CV has R^2 of 0.9999999. Alpha is: 0.0005. # folds are: 4.
Lasso CV has R^2 of 0.9999999. Alpha is: 0.0005. # folds are: 5.
Lasso CV has R^2 of 0.9999999. Alpha is: 0.0005. # folds are: 7.
Lasso CV has R^2 of 0.9999996. Alpha is: 0.001. # folds are: 9.
Lasso CV has R^2 of 0.9999996. Alpha is: 0.001. # folds are: 10.
Lasso CV has R^2 of 0.9999999. Alpha is: 0.0005. # folds are: 12.
Lasso CV has R^2 of 0.9999996. Alpha is: 0.001. # folds are: 15.


Gotta ask about this one. Actually don't have a clue.

In [0]:
# Problem 3
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression

# generate some sparse data to play with
np.random.seed(7)

n_samples, n_features = 100, 200
X = np.random.randn(n_samples, n_features)

k = 5
# beta generated with k nonzeros
#coef = 10 * np.random.randn(n_features)
coef = 10 * np.ones(n_features)
inds = np.arange(n_features)
# print("coef\n", coef)
# print("inds\n", inds)

np.random.shuffle(inds)
coef[inds[k:]] = 0  # sparsify coef
# print("ind\n", inds)
# print("coef after shuffling\n", coef)
y = np.dot(X, coef).reshape(n_samples, 1)

# add noise
y += 0.01 * np.random.randn(n_samples, 1)
# print("actual y\n", y)


# implement k-fold cross-validation
# split the data up into k subsets
k_values = [1.2, 1.5, 2, 5, 10, 50, 100]
num_of_trials = 50
beta_star = coef
nonzero_betas = [15, 18, 34, 51, 78]
betas = np.empty((200, 0))


for k in k_values:
  for i in range(num_of_trials):
    # regenerate y to get different noise
    y = np.dot(X, coef).reshape(n_samples, 1)
    # add noise
    y += 0.01 * np.random.randn(n_samples, 1)
    
    testSize = 1/k
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testSize)

    lasso = Lasso()
    lasso.fit(X_train, y_train)
    train_score = lasso.score(X_train, y_train)
    test_score = lasso.score(X_test, y_test)
    coeff_used = np.sum(lasso.coef_!=0)
    
    betas = np.append(betas, lasso.coef_.reshape(200, 1), axis=1)
    
  # calculate bias  
  beta_means = np.mean(betas, axis=1)
  bias = beta_star.reshape(200) - beta_means
  total_bias = np.sum(np.square(bias))
  
  # calculate variance
  beta_var = np.var(betas, axis=1)
  avg_var = np.sum(beta_var)/200
  # print(betas, "\n\n")
  
  print(f"For k = {k}")
  for beta in nonzero_betas:
    print(f"for feature {k}, bias = {bias[beta]}, variance = {beta_var[beta]}")  
  print(f"total bias is {total_bias}")
  print(f"average variance = {avg_var}\n")
  
  # print(errors.shape)
  # print(f"calculated bias is {np.mean(errors, axis=1)}")
  
  
  betas = np.empty((200, 0))
    
    

For k = 1.2
for feature 1.2, bias = 8.466428248155001, variance = 4.71841880886136
for feature 1.2, bias = 7.919714858757581, variance = 10.275729524512737
for feature 1.2, bias = 8.72803310316235, variance = 5.629021776035108
for feature 1.2, bias = 8.477316147391047, variance = 8.959620511238011
for feature 1.2, bias = 8.910355885810182, variance = 5.823641417628076
total bias is 365.3960188057588
average variance = 0.5712874172860043

(0, 0)
For k = 1.5
for feature 1.5, bias = 3.2826642178462535, variance = 6.80632492468971
for feature 1.5, bias = 2.7332342638012843, variance = 6.311095501238107
for feature 1.5, bias = 3.781941580252318, variance = 8.644463919989093
for feature 1.5, bias = 3.30716152351148, variance = 7.717223129665692
for feature 1.5, bias = 3.2663474376082977, variance = 8.127489499673894
total bias is 54.75064613404826
average variance = 0.24383361277874838

(200, 0)
For k = 2
for feature 2, bias = 1.255560283188622, variance = 0.10144424487246839
for feature 2, 